## Felix's Contribution (You can also start running after this block)

Varying the mem parameters using the domain parameter to get the domain of the values.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.status()
Pkg.instantiate()
Pkg.update()
Pkg.status()

  Activating project at `~/OptimizationParameterTuning/Julia Notebook`


Status `~/OptimizationParameterTuning/Julia Notebook/Project.toml`
  [54578032] ADNLPModels v0.8.13
  [6e4b80f9] BenchmarkTools v1.6.0
  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.8.0
  [7073ff75] IJulia v1.30.4
  [10dff2fc] JSOSolvers v0.14.3
  [ed6ae0be] JSOSuite v0.3.0
  [a4795742] NLPModels v0.21.5
  [5049e819] OptimizationProblems v0.9.2
  [ff4d7338] SolverCore v0.3.8
  [d076d87d] SolverParameters v0.1.2


    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/OptimizationParameterTuning/Julia Notebook/Project.toml`
  No Changes to `~/OptimizationParameterTuning/Julia Notebook/Manifest.toml`


Status `~/OptimizationParameterTuning/Julia Notebook/Project.toml`
  [54578032] ADNLPModels v0.8.13
  [6e4b80f9] BenchmarkTools v1.6.0
  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.8.0
  [7073ff75] IJulia v1.30.4
  [10dff2fc] JSOSolvers v0.14.3
  [ed6ae0be] JSOSuite v0.3.0
  [a4795742] NLPModels v0.21.5
  [5049e819] OptimizationProblems v0.9.2
  [ff4d7338] SolverCore v0.3.8
  [d076d87d] SolverParameters v0.1.2


In [ ]:
using BenchmarkTools
using CSV, DataFrames
using Random
using OptimizationProblems
using OptimizationProblems.ADNLPProblems # https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl
                                         # contains a test set of problems of type ADNLPModel
using NLPModels # see https://github.com/JuliaSmoothOptimizers/NLPModels.jl, it defines an abstract API to access a continuous optimization problem
using ADNLPModels # see https://github.com/JuliaSmoothOptimizers/ADNLPModels.jl
                  # is a concrete implementation of the ty
using SolverParameters # Define the main structure to handle the algorithm's parameters
using JSOSolvers

nlp = OptimizationProblems.ADNLPProblems.arglina(matrix_free = true) # is one of them
meta = OptimizationProblems.meta
problem_names = meta[meta.contype .== :unconstrained .&& .!meta.has_bounds .&& meta.nvar .>= 5, :name]; # LBFGS is an algorithm to solve unconstrained problems
                                                                   # For this example, we select only problems of size up to 10.
problems = [Meta.parse("OptimizationProblems.ADNLPProblems.eval($problem)()") for problem ∈ problem_names]; # https://jso.dev/OptimizationProblems.jl/dev/benchmark/                                                
filename = "./results/lbfgs.csv"
df = DataFrame(
                :status => Symbol[],
                :name => String[],
                :solver => String[],
                :mem => Int[],
                :nvar => Int[],
                :time => Float64[],
                :memory => Float64[],
                :nvmops => Int[],
                :neval_obj => Int[],
                :neval_grad => Int[]
            )


# load from CSV file
completed = Set{Tuple{String, String, Int}}()
if isfile(filename) && filesize(filename) > 0
    df_completed = CSV.read(filename, DataFrame)
    completed = Set(zip(df_completed.name, df_completed.solver, df_completed.mem))
end

i = 0
for pb_expr in problems
    nlp = eval(pb_expr)
    i += 1
    param_set = JSOSolvers.LBFGSParameterSet(nlp)
    r = domain(param_set.mem)
    problem = nlp.meta.name
    solver = "LBFGSSolver"
    for mem in r.lower:r.upper
        key = (problem, solver, mem)
        if key in completed
            @info "Skip $key — already seen"
            continue
        else
            reset!(nlp)
            println("Running $pb_expr with mem=$mem")
            try
                bench = @benchmark JSOSolvers.lbfgs($nlp; mem = $mem)
                stats = JSOSolvers.lbfgs(nlp; mem = mem)
                push!(df, (
                            status = stats.status, 
                            name = problem,
                            solver = solver,
                            mem = mem,
                            nvar = nlp.meta.nvar, 
                            time = minimum(bench).time,
                            memory = minimum(bench).memory, 
                            nvmops = stats.solver_specific[:nprod],
                            neval_obj = nlp.counters.neval_obj,
                            neval_grad = nlp.counters.neval_grad
                        )
                    )
                push!(completed, (problem, solver, mem))
            catch e
                @info "Solver failed on $(nlp.meta.name): $e"
                break
            end
        end
        CSV.write(filename, DataFrame([last(df)]); append=true)
    end
end


┌ Info: Skip ("NZF1", "LBFGSSolver", 1) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 2) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51


Running (OptimizationProblems.ADNLPProblems.eval(NZF1))() with mem=9


┌ Info: Skip ("NZF1", "LBFGSSolver", 3) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 4) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 5) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 6) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 7) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90Z

Running (OptimizationProblems.ADNLPProblems.eval(NZF1))() with mem=53
Running (OptimizationProblems.ADNLPProblems.eval(NZF1))() with mem=54


┌ Info: Skip ("NZF1", "LBFGSSolver", 12) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 13) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 14) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 15) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:51
┌ Info: Skip ("NZF1", "LBFGSSolver", 16) — already seen
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJl

...

Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=1
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=2
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=3
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=4
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=5
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=6
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=7
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=8
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=9
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=10
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=11
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=12
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=13
Running (OptimizationProblems.ADNLPProblems.eval(cosine))() with mem=14
R

┌ Info: Solver failed on cragglvy: ArgumentError("string too large (8) to convert to String7")
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:74


Running (OptimizationProblems.ADNLPProblems.eval(cragglvy2))() with mem=1


┌ Info: Solver failed on cragglvy2: ArgumentError("string too large (9) to convert to String7")
└ @ Main /home/gaoyuc10/OptimizationParameterTuning/Julia Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sdnNjb2RlLXJlbW90ZQ==.jl:74


Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=1
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=2
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=3
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=4
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=5
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=6
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=7
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=8
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=9
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=10
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=11
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=12
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=13
Running (OptimizationProblems.ADNLPProblems.eval(curly))() with mem=14
Running (Optimi